Hands-on Lab: Acquiring and Processing Information on the World's Largest Banks

Estimated Time: 60 mins
In this project, you will put all the skills acquired throughout the course and your knowledge of basic Python to test. You will work on real-world data and perform the operations of Extraction, Transformation, and Loading (ETL) as required.

Disclaimer:

Cloud IDE is not a persistent platform, and you will lose your progress every time you restart this lab. We recommend saving a copy of your file on your local machine as a protective measure against data loss.

Project Scenario:
You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

Particulars of the code to be made have been shared below.

Parameter	Value
Code name	banks_project.py
Data URL	https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks
Exchange rate CSV path	https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Table Attributes (upon Extraction only)	Name, MC_USD_Billion
Table Attributes (final)	Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion
Output CSV Path	./Largest_banks_data.csv
Database name	Banks.db
Table name	Largest_banks
Log file	code_log.txt
Project tasks
Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

Task 2:
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.

Task 3:
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.

Task 4:
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

Task 5:
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

Task 6:
Run queries on the database table. Write a function run_queries(), execute a given set of queries and verify the output.

Task 7:
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

In [1]:
from datetime import datetime

import requests
from lxml import html
import pandas as pd

import sqlite3

In [2]:
def log_progress(message, log_file='code_log.txt'):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(log_file, 'a', encoding='utf-8') as f:
        f.write(f"{timestamp} - {message}\n")

In [3]:
def extract(url):

    log_progress("Veri çekme işlemi başladı.")

    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    doc = html.fromstring(response.content)
    
    table = doc.xpath('//table[contains(@class, "wikitable") and contains(@class, "sortable") and contains(@class, "mw-collapsible")]')[0]
    rows = table.xpath('.//tbody/tr')
    
    data = []
    
    for row in rows:
        try:
            name_cell = row.xpath('./td[2]/a/text()')
            name = name_cell[0].strip() if name_cell else None
            
            market_cap_cell = row.xpath('./td[3]/text()')
            market_cap_raw = market_cap_cell[0].strip() if market_cap_cell else None
            
            if name and market_cap_raw:
                market_cap = float(market_cap_raw.replace(',', '').strip())
                data.append({'Name': name, 'MC_USD_Billion': market_cap})
        except Exception:
            continue  # Hatalı satırları atla
    
    # DataFrame oluştur
    df = pd.DataFrame(data)
    
    # Index'i 1'den başlat
    df.index = range(1, len(df) + 1)

    log_progress("Veri çekme işlemi tamamlandı.")
    return df

In [4]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
df_extracted = extract(url)
df_extracted.head(10)


,Name,MC_USD_Billion
1,JPMorgan Chase,432.92
2,Bank of America,231.52
3,Industrial and Commercial Bank of China,194.56
4,Agricultural Bank of China,160.68
5,HDFC Bank,157.91
6,Wells Fargo,155.87
7,HSBC Holdings PLC,148.90
8,Morgan Stanley,140.83
9,China Construction Bank,139.82
10,Bank of China,136.81


In [5]:
!curl -o exchange_rate.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    45  100    45    0     0     62      0 --:--:-- --:--:-- --:--:--    62


In [6]:
def transform(df, exchange_csv_path="exchange_rate.csv"):
    # Döviz kuru dosyasını oku
    exchange_df = pd.read_csv(exchange_csv_path, index_col="Currency")
    
    # Kurları çek
    gbp_rate = exchange_df.loc["GBP", "Rate"]
    eur_rate = exchange_df.loc["EUR", "Rate"]
    inr_rate = exchange_df.loc["INR", "Rate"]
    
    # USD -> GBP, EUR, INR dönüşümleri
    df["MC_USD_Billion"] = df["MC_USD_Billion"].astype(float)
    df["MC_GBP_Billion"] = (df["MC_USD_Billion"] * gbp_rate).round(2)
    df["MC_EUR_Billion"] = (df["MC_USD_Billion"] * eur_rate).round(2)
    df["MC_INR_Billion"] = (df["MC_USD_Billion"] * inr_rate).round(2)
    
    return df


In [7]:
df_transformed = transform(df_extracted)


In [8]:
def load_to_csv(df, output_path="Largest_banks_data.csv"):
    df.to_csv(output_path, index=False, encoding='utf-8')
    

In [9]:
def load_to_db(df, db_name="Banks.db", table_name="Largest_banks"):
    conn = sqlite3.connect(db_name)
    log_progress("SQL Connection initiated")
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    log_progress("Data loaded to Database as a table, Executing queries")

    # Basit sorgu: MC_USD_Billion > 300
    query = f"SELECT Name, MC_USD_Billion FROM {table_name} WHERE MC_USD_Billion > 300 ORDER BY MC_USD_Billion DESC"
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()

    print("\nBanks with Market Cap > 300 Billion USD:\n")
    for row in results:
        print(row)

    log_progress("Process Complete")
    
    conn.close()


In [10]:
if __name__ == "__main__":
    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"

    log_progress("Preliminaries complete. Initiating ETL process")

    log_progress("Data extraction started.")
    df_extracted = extract(url)
    log_progress("Data extraction complete. Initiating Transformation process")

    # 2. Transform
    log_progress("Data transformation started.")
    df_transformed = transform(df_extracted)
    log_progress("Data transformation complete. Initiating Loading process")


    # 3. Load to CSV
    load_to_csv(df_transformed)
    log_progress("Data saved to CSV file")

    
    # 4. Load to DB and run query
    log_progress("SQL Connection initiated")
    load_to_db(df_transformed)
    log_progress("Server Connection closed")


Banks with Market Cap > 300 Billion USD:

('JPMorgan Chase', 432.92)
